In [ ]:
!pip install git+https://github.com/mathLab/EZyRB
!mkdir data
!wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_coord.npy
!wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_mu.npy
!wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_snapshots.npy
!wget -P data https://github.com/giovastabile/roma_phd_course_exercises/raw/master/LESSON_1/data/tut1_triangles.npy

In [ ]:


import numpy as np
import matplotlib.tri as mtri
import matplotlib.pyplot as plt

from ezyrb import POD, RBF, Database
from ezyrb import ReducedOrderModel as ROM
import numpy.linalg as LA

from scipy.spatial import Delaunay

# ## Offline phase
# 
# In the *offline* phase, we need some samples of the parametric high-fidelity model. In this case, we extract 8 snapshots from the numerical model implemented in **FEniCS**, and we import them and the related parameters.


snapshots = np.load('data/tut1_snapshots.npy')
param = np.load('data/tut1_mu.npy')

print(snapshots.shape, param.shape)

train_snapshots = snapshots[0:8,:]
train_param = param[0:8,:]
print(train_snapshots.shape, train_param.shape)

# Moreover, to visualize the solution (both the higher-order one and the reduced one), we import also the mesh information to be able to create the triangulation. We underline this additional step is related only to plotting purpose, and not mandatory for the reduced space generation.


tri = np.load('data/tut1_triangles.npy')
coord = np.load('data/tut1_coord.npy')
triang = mtri.Triangulation(coord[0],coord[1],tri)


# For the sake of clarity the snapshots are plotted.


fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(16, 6), sharey=True, sharex=True)
ax = ax.flatten()
for i in range(8):
    ax[i].triplot(triang, 'b-', lw=0.1)
    cm = ax[i].tripcolor(triang, snapshots[i])
    fig.colorbar(cm, ax=ax[i])
    ax[i].set_title('($\mu_0={:5.2f}, \mu_1={:5.2f})$'.format(*param[i]))

plt.show()
# First of all, we create a `Database` object from the parameters and the snapshots.

db = Database(train_param, train_snapshots)



# Then we need a reduction object. In this case we use the proper orthogonal decomposition so we create a `POD` object. We use here all the default parameters, but for the complete list of available arguments we refer to original documentation of [POD](https://mathlab.github.io/EZyRB/pod.html) class.

pod = POD('svd')


# Then we instantiate the `RBF` class for interpolating the solution manifold. Also in this case, [RBF](https://mathlab.github.io/EZyRB/rbf.html) documentation is the perfect starting point to explore such class.


rbf = RBF()


# Few lines of code and our reduced model is created!
# To complete everything, we create the `ReducedOrderModel` (aliased to `ROM` in this tutorial) object by passing the already created objects. For clarity, we puntualize that we need to pass the **instances** and not the classes. Simply changing such line (with different objects) allows to test different frameworks in a very modular way.
# The `fit()` function computes the reduced model, meaning that the original snapshots in the database are projected onto the POD space and the RBF interpolator is created.

rom = ROM(db, pod, rbf)
rom.fit();



plt.plot(train_param[:,0],train_param[:,1],'*')

plt.title('Train vs test parameter location')
plt.grid()
plt.show()
# ## Error Approximation & Improvement
# 
# At the moment, we used a database which is composed by 8 files. we would have an idea of the approximation accuracy we are able to reach with these high-fidelity solutions. Using the *leave-one-out* strategy, an error is computed for each parametric point in our database and these values are returned as array. 


sample_space_tria = Delaunay(train_param)
loo_error = []
print('Leave one out error')
for pt, error in zip(rom.database.parameters, rom.loo_error()):
    print(pt, error)
    loo_error.append(error)
    
error = np.array(loo_error)
    



# Moreover, we can use the information about the errors to locate the parametric points where we have to compute the new high-fidelity solutions and add these to the database in order to optimally improve the accuracy.


mu_opt = rom.optimal_mu()
print('Optimal parameter for enrichment')
print(mu_opt)

ps_triang = mtri.Triangulation(train_param[:,0],train_param[:,1],sample_space_tria.simplices)
plt.triplot(ps_triang, 'b-', lw=0.1)
plt.tripcolor(ps_triang, error)
plt.plot(train_param[:,0],train_param[:,1],'*')
plt.plot(mu_opt[:,0],mu_opt[:,1],'*')
plt.colorbar()
plt.title('Leave one out error')
plt.grid()
plt.show()

# These function can be used to achieve the wanted (estimated) accuracy.
